In [1]:
from sklearn import metrics,model_selection,preprocessing,linear_model,ensemble,decomposition,tree
from sklearn.metrics import mean_squared_error,mean_absolute_error, roc_auc_score,f1_score,roc_curve,accuracy_score,r2_score
from sklearn.preprocessing import StandardScaler, RobustScaler,MinMaxScaler,PolynomialFeatures
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder,OrdinalEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split, KFold ,GridSearchCV,StratifiedKFold,RepeatedKFold
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,HistGradientBoostingRegressor
from sklearn.neural_network import MLPRegressor  # Import the MLPRegressor
from catboost import CatBoostRegressor,CatBoostClassifier
from lightgbm import LGBMRegressor,LGBMClassifier
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor,XGBClassifier
from tabulate import tabulate


import geopandas as gpd
import pandas as pd
import folium
#!pip install geopy
#from geopy.geocoders import Nominatim

import gc,itertools, pickle, re,time,warnings
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
from random import choice, choices
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
#pip install pandas-profiling
#import pandas_profiling
import seaborn as sns
%matplotlib inline
import numpy as np
import json
import os

from functools import reduce
from tqdm import tqdm
from itertools import cycle
from matplotlib_venn import venn2

from scipy import stats
from functools import reduce
seed = 777
import random
random.seed(seed)
np.random.seed(seed)
plt.style.use("ggplot")
color_pal = plt.rcParams["axes.prop_cycle"].by_key()["color"]

def calculate_rmse(true_values, predicted_values):
    mse = mean_squared_error(true_values, predicted_values)
    rmse = np.sqrt(mse)
    return rmse

def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

#from google.colab import files, drive
#drive.mount('./content')

In [2]:
# Load files
#data_path = "/content/content/MyDrive/DATA-ANALYSIS/ZINDI-COMPETITIONS/TABULAR/Digital Green Crop Yield Estimate Challeng/"
train = pd.read_csv('train_df.csv')
test = pd.read_csv('test_df.csv')
#sample_submission = pd.read_csv(data_path + 'SampleSubmission.csv')
#var_desc = pd.read_csv(data_path + 'VariableDescription.csv')

In [3]:
#var_desc

In [4]:
dummy_columns = ["P", "K", "Mg", "pH"]
train.drop(columns=dummy_columns+['sample_index'],inplace=True)
print(train.shape)
train.head()

(13856, 2402)


,Target,Value,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,feat10,feat11,feat12,feat13,feat14,feat15,feat16,feat17,feat18,feat19,feat20,feat21,feat22,feat23,feat24,feat25,feat26,feat27,feat28,feat29,feat30,feat31,feat32,feat33,feat34,feat35,feat36,feat37,feat38,feat39,feat40,feat41,feat42,feat43,feat44,feat45,feat46,feat47,feat48,...,feat2351,feat2352,feat2353,feat2354,feat2355,feat2356,feat2357,feat2358,feat2359,feat2360,feat2361,feat2362,feat2363,feat2364,feat2365,feat2366,feat2367,feat2368,feat2369,feat2370,feat2371,feat2372,feat2373,feat2374,feat2375,feat2376,feat2377,feat2378,feat2379,feat2380,feat2381,feat2382,feat2383,feat2384,feat2385,feat2386,feat2387,feat2388,feat2389,feat2390,feat2391,feat2392,feat2393,feat2394,feat2395,feat2396,feat2397,feat2398,feat2399,feat2400
0,0_P,45.1,0.203683,0.203992,0.202177,0.205636,0.210894,0.220505,0.226073,0.227012,0.228581,0.230678,0.233982,0.238588,0.241646,0.250014,0.258405,0.261899,0.266396,0.270737,0.274890,0.273968,0.276430,0.283327,0.295028,0.308494,0.316209,0.320717,0.327333,0.331691,0.337992,0.342740,0.349642,0.355668,0.360805,0.364883,0.367570,0.370319,0.376717,0.381299,0.387101,0.393264,0.399336,0.402669,0.405688,0.407629,0.411576,0.416467,0.419428,0.419526,...,1.080411,1.095705,1.129028,1.153727,1.191501,1.206518,1.240862,1.266370,1.281632,1.303487,1.320699,1.343543,1.349235,1.355927,1.355282,1.360762,1.357333,1.372652,1.387570,1.400359,0.574390,0.584549,0.670901,0.769619,0.920841,1.042641,1.146396,1.211927,1.273028,1.312649,1.513034,1.593172,1.678123,1.751818,1.816820,1.870275,1.911317,1.919118,1.924227,1.962802,0.805311,0.936756,1.290456,1.626534,1.777969,2.094892,2.382082,2.567942,2.697733,2.733829
1,1_P,44.8,0.250603,0.249785,0.246786,0.250632,0.255035,0.265170,0.270217,0.269648,0.269498,0.270211,0.272503,0.276662,0.279905,0.288868,0.297204,0.301549,0.306996,0.313169,0.319590,0.320913,0.326546,0.336216,0.350023,0.364886,0.372914,0.377352,0.383670,0.387081,0.392847,0.396461,0.401595,0.405064,0.406857,0.406138,0.403846,0.402338,0.404513,0.405366,0.408354,0.412235,0.416498,0.418540,0.420223,0.421406,0.424962,0.428652,0.430207,0.429708,...,1.256190,1.502146,1.758616,2.011209,2.160844,2.194579,2.249652,2.285842,2.307155,2.331671,2.351600,2.378405,2.375424,2.375780,2.363581,2.404702,2.419109,2.419744,2.425575,2.429126,0.706865,0.714469,0.781618,0.895926,1.084361,1.143641,1.187382,1.231635,1.280067,1.232515,1.738577,2.511333,3.038777,3.175525,3.269756,3.328903,3.365557,3.357815,3.421406,3.428380,0.992393,1.105412,1.483055,1.721859,1.699395,2.483675,4.277375,4.606743,4.764096,4.809254
2,2_P,44.4,0.191200,0.189831,0.187725,0.191629,0.196252,0.205140,0.209914,0.210256,0.210999,0.211794,0.214068,0.218355,0.221650,0.229133,0.235590,0.238543,0.242081,0.245498,0.248445,0.247065,0.249292,0.254473,0.263325,0.273994,0.279563,0.283124,0.288288,0.291484,0.296931,0.301143,0.306834,0.311672,0.315907,0.318279,0.319126,0.320443,0.324220,0.326946,0.331071,0.335602,0.340231,0.343282,0.345376,0.346862,0.350116,0.353510,0.355157,0.355238,...,0.866659,0.868626,0.886784,0.897235,0.921281,0.942381,0.972106,0.993024,1.004513,1.016845,1.026514,1.040119,1.028430,1.024554,1.021568,1.045355,1.059899,1.062368,1.069155,1.073775,0.539424,0.545641,0.616830,0.695538,0.813433,0.904103,0.975803,1.019546,1.059941,1.061241,1.211937,1.250024,1.300998,1.373126,1.422106,1.455332,1.458446,1.450438,1.497318,1.510940,0.756372,0.862336,1.140623,1.386935,1.473936,1.671600,1.852783,2.015047,2.058019,2.104777
3,3_P,46.5,0.275959,0.276271,0.273568,0.278322,0.284468,0.296392,0.303449,0.305312,0.308380,0.311888,0.317236,0.324387,0.329835,0.341946,0.353035,0.358912,0.365899,0.372545,0.378533,0.377787,0.381994,0.391503,0.406517,0.423829,0.433760,0.440477,0.450120,0.456161,0.465329,0.471999,0.481131,0.488511,0.494952,0.499083,0.501164,0.503961,0.511077,0.516137,0.523359,0.530998,0.538473,0.542569,0.545731,0.547505,0.552389,0.557519,0.560445,0.560457,...,1.402975,1.377883,1.377740,1.365293,1.391699,1.412411,1.447749,1.48

In [5]:
print(test.shape)
test.head(3)

(4616, 2402)


,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,feat10,feat11,feat12,feat13,feat14,feat15,feat16,feat17,feat18,feat19,feat20,feat21,feat22,feat23,feat24,feat25,feat26,feat27,feat28,feat29,feat30,feat31,feat32,feat33,feat34,feat35,feat36,feat37,feat38,feat39,feat40,feat41,feat42,feat43,feat44,feat45,feat46,feat47,feat48,feat49,feat50,...,feat2353,feat2354,feat2355,feat2356,feat2357,feat2358,feat2359,feat2360,feat2361,feat2362,feat2363,feat2364,feat2365,feat2366,feat2367,feat2368,feat2369,feat2370,feat2371,feat2372,feat2373,feat2374,feat2375,feat2376,feat2377,feat2378,feat2379,feat2380,feat2381,feat2382,feat2383,feat2384,feat2385,feat2386,feat2387,feat2388,feat2389,feat2390,feat2391,feat2392,feat2393,feat2394,feat2395,feat2396,feat2397,feat2398,feat2399,feat2400,Target,mean_norm
0,0.173565,0.171952,0.168504,0.170999,0.173946,0.181525,0.185026,0.184056,0.183710,0.183698,0.184985,0.187467,0.189658,0.196452,0.202846,0.207412,0.213938,0.222476,0.232073,0.238128,0.247958,0.260854,0.276417,0.291528,0.300351,0.305525,0.311848,0.315623,0.320844,0.323720,0.327251,0.327792,0.325899,0.320993,0.314128,0.308204,0.305358,0.301816,0.300822,0.301477,0.303310,0.303769,0.304400,0.304489,0.306138,0.307097,0.305769,0.302423,0.300566,0.299315,...,1.688006,2.042916,2.297939,2.377649,2.466454,2.507972,2.530190,2.559515,2.581921,2.615295,2.617831,2.625087,2.613590,2.660765,2.680289,2.683594,2.687329,2.684616,0.489267,0.491960,0.528080,0.650628,0.887739,0.887057,0.858053,0.846928,0.843302,0.756928,1.354611,2.418203,3.218510,3.481309,3.586095,3.656175,3.709706,3.713505,3.791356,3.797913,0.688768,0.754274,1.174493,1.269823,1.064050,1.963784,4.521961,5.065660,5.246762,5.330273,0_P,70.302656
1,0.202796,0.201845,0.199020,0.202500,0.206132,0.214863,0.219335,0.218668,0.218889,0.219440,0.221122,0.224898,0.227561,0.236100,0.243862,0.248254,0.253956,0.260094,0.266156,0.267235,0.272653,0.281744,0.294286,0.307994,0.314845,0.318900,0.324794,0.327503,0.332724,0.336211,0.341693,0.345040,0.347407,0.347685,0.346067,0.344727,0.346363,0.347024,0.349843,0.353515,0.357605,0.359821,0.362217,0.363633,0.367676,0.371853,0.374272,0.374649,0.376807,0.379905,...,1.798136,2.096276,2.280492,2.336384,2.416731,2.466844,2.495732,2.528185,2.554744,2.590548,2.593327,2.601961,2.589237,2.643366,2.655447,2.637913,2.626337,2.615737,0.571762,0.577881,0.635002,0.744523,0.917573,0.976870,1.023253,1.082452,1.146555,1.090484,1.659797,2.571110,3.205332,3.410100,3.538129,3.616678,3.675339,3.682659,3.754580,3.710804,0.800777,0.901758,1.248479,1.495482,1.494265,2.381440,4.517328,4.982208,5.206562,5.262480,1_P,70.302656
2,0.164810,0.163459,0.161074,0.163151,0.166365,0.173361,0.176534,0.175363,0.174817,0.174266,0.175142,0.177567,0.179800,0.186686,0.194100,0.199420,0.207205,0.216406,0.226819,0.234233,0.245464,0.259351,0.275583,0.291327,0.300969,0.306700,0.313685,0.318030,0.323988,0.326964,0.330085,0.330050,0.327086,0.321105,0.313493,0.306747,0.303655,0.299919,0.298985,0.299455,0.300853,0.300812,0.300627,0.300261,0.301276,0.301984,0.300389,0.296479,0.293648,0.291786,...,1.989744,2.470337,2.807123,2.898035,3.001448,3.048183,3.074964,3.106142,3.131777,3.165942,3.161512,3.161910,3.150329,3.199207,3.225819,3.235964,3.238828,3.231759,0.464090,0.469189,0.501251,0.635263,0.894274,0.885812,0.848520,0.821781,0.808230,0.666904,1.421769,2.859085,3.925901,4.236832,4.356789,4.433868,4.480423,4.470493,4.564563,4.577605,0.653497,0.721019,1.169714,1.269090,0.967839,2.094396,5.501404,6.153006,6.340413,6.413906,2_P,70.302656


In [6]:
#print(sample_submission.shape)
#sample_submission.head()

In [7]:
test['Param'] = test['Target'].str.split('_').str.get(1)
train['Param'] = train['Target'].str.split('_').str.get(1)
test.head()

,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,feat10,feat11,feat12,feat13,feat14,feat15,feat16,feat17,feat18,feat19,feat20,feat21,feat22,feat23,feat24,feat25,feat26,feat27,feat28,feat29,feat30,feat31,feat32,feat33,feat34,feat35,feat36,feat37,feat38,feat39,feat40,feat41,feat42,feat43,feat44,feat45,feat46,feat47,feat48,feat49,feat50,...,feat2354,feat2355,feat2356,feat2357,feat2358,feat2359,feat2360,feat2361,feat2362,feat2363,feat2364,feat2365,feat2366,feat2367,feat2368,feat2369,feat2370,feat2371,feat2372,feat2373,feat2374,feat2375,feat2376,feat2377,feat2378,feat2379,feat2380,feat2381,feat2382,feat2383,feat2384,feat2385,feat2386,feat2387,feat2388,feat2389,feat2390,feat2391,feat2392,feat2393,feat2394,feat2395,feat2396,feat2397,feat2398,feat2399,feat2400,Target,mean_norm,Param
0,0.173565,0.171952,0.168504,0.170999,0.173946,0.181525,0.185026,0.184056,0.183710,0.183698,0.184985,0.187467,0.189658,0.196452,0.202846,0.207412,0.213938,0.222476,0.232073,0.238128,0.247958,0.260854,0.276417,0.291528,0.300351,0.305525,0.311848,0.315623,0.320844,0.323720,0.327251,0.327792,0.325899,0.320993,0.314128,0.308204,0.305358,0.301816,0.300822,0.301477,0.303310,0.303769,0.304400,0.304489,0.306138,0.307097,0.305769,0.302423,0.300566,0.299315,...,2.042916,2.297939,2.377649,2.466454,2.507972,2.530190,2.559515,2.581921,2.615295,2.617831,2.625087,2.613590,2.660765,2.680289,2.683594,2.687329,2.684616,0.489267,0.491960,0.528080,0.650628,0.887739,0.887057,0.858053,0.846928,0.843302,0.756928,1.354611,2.418203,3.218510,3.481309,3.586095,3.656175,3.709706,3.713505,3.791356,3.797913,0.688768,0.754274,1.174493,1.269823,1.064050,1.963784,4.521961,5.065660,5.246762,5.330273,0_P,70.302656,P
1,0.202796,0.201845,0.199020,0.202500,0.206132,0.214863,0.219335,0.218668,0.218889,0.219440,0.221122,0.224898,0.227561,0.236100,0.243862,0.248254,0.253956,0.260094,0.266156,0.267235,0.272653,0.281744,0.294286,0.307994,0.314845,0.318900,0.324794,0.327503,0.332724,0.336211,0.341693,0.345040,0.347407,0.347685,0.346067,0.344727,0.346363,0.347024,0.349843,0.353515,0.357605,0.359821,0.362217,0.363633,0.367676,0.371853,0.374272,0.374649,0.376807,0.379905,...,2.096276,2.280492,2.336384,2.416731,2.466844,2.495732,2.528185,2.554744,2.590548,2.593327,2.601961,2.589237,2.643366,2.655447,2.637913,2.626337,2.615737,0.571762,0.577881,0.635002,0.744523,0.917573,0.976870,1.023253,1.082452,1.146555,1.090484,1.659797,2.571110,3.205332,3.410100,3.538129,3.616678,3.675339,3.682659,3.754580,3.710804,0.800777,0.901758,1.248479,1.495482,1.494265,2.381440,4.517328,4.982208,5.206562,5.262480,1_P,70.302656,P
2,0.164810,0.163459,0.161074,0.163151,0.166365,0.173361,0.176534,0.175363,0.174817,0.174266,0.175142,0.177567,0.179800,0.186686,0.194100,0.199420,0.207205,0.216406,0.226819,0.234233,0.245464,0.259351,0.275583,0.291327,0.300969,0.306700,0.313685,0.318030,0.323988,0.326964,0.330085,0.330050,0.327086,0.321105,0.313493,0.306747,0.303655,0.299919,0.298985,0.299455,0.300853,0.300812,0.300627,0.300261,0.301276,0.301984,0.300389,0.296479,0.293648,0.291786,...,2.470337,2.807123,2.898035,3.001448,3.048183,3.074964,3.106142,3.131777,3.165942,3.161512,3.161910,3.150329,3.199207,3.225819,3.235964,3.238828,3.231759,0.464090,0.469189,0.501251,0.635263,0.894274,0.885812,0.848520,0.821781,0.808230,0.666904,1.421769,2.859085,3.925901,4.236832,4.356789,4.433868,4.480423,4.470493,4.564563,4.577605,0.653497,0.721019,1.169714,1.269090,0.967839,2.094396,5.501404,6.153006,6.340413,6.413906,2_P,70.302656,P
3,0.196746,0.195933,0.192789,0.195939,0.199449,0.207141,0.210958,0.209670,0.209044,0.208800,0.209746,0.212316,0.214074,0.221122,0.227590,0.231175,0.236460,0.242244,0.248283,0.249548,0.254676,0.263047,0.274811,0.286976,0.292836,0.295371,0.299582,0.301323,0.305262,0.307263,0.310883,0.312774,0.313215,0.311683,0.309021,0.307060,0.308388,0.308638,0.310459,0.313198,0.316359,0.317897,0.319637,0.321255,0.325305,0.329748,0.332382,0.332927,0.335776,0.339599,...,2.703383,3.003249,3.069666,3.164584,3.224090,3.256872,3.295791,3.32

In [8]:
train.groupby(['Param'])['Value'].nunique()

Param
K     2020
Mg    1983
P     2420
pH    1823
Name: Value, dtype: int64

## Scale Columns??

In [9]:
# Encode the 'period_of_day' column
#combined_df['period_of_day'] = label_encoder.fit_transform(combined_df['period_of_day'])

#scale columns
def scale_columns(df):
    scaler = MinMaxScaler()
    #scaler = StandardScaler()
    #scaler = RobustScaler()
    cols_to_exclude = ["Target","mean_norm","Param","Value"] 
    cols_to_scale = [col for col in df.columns if col not in cols_to_exclude]
    scaled_values = scaler.fit_transform(df[cols_to_scale])
    scaled_df = df.copy()
    scaled_df[cols_to_scale] = scaled_values
    return scaled_df
#train= scale_columns(train)
#test=scale_columns(test)
train.head(3)#"""

,Target,Value,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,feat10,feat11,feat12,feat13,feat14,feat15,feat16,feat17,feat18,feat19,feat20,feat21,feat22,feat23,feat24,feat25,feat26,feat27,feat28,feat29,feat30,feat31,feat32,feat33,feat34,feat35,feat36,feat37,feat38,feat39,feat40,feat41,feat42,feat43,feat44,feat45,feat46,feat47,feat48,...,feat2352,feat2353,feat2354,feat2355,feat2356,feat2357,feat2358,feat2359,feat2360,feat2361,feat2362,feat2363,feat2364,feat2365,feat2366,feat2367,feat2368,feat2369,feat2370,feat2371,feat2372,feat2373,feat2374,feat2375,feat2376,feat2377,feat2378,feat2379,feat2380,feat2381,feat2382,feat2383,feat2384,feat2385,feat2386,feat2387,feat2388,feat2389,feat2390,feat2391,feat2392,feat2393,feat2394,feat2395,feat2396,feat2397,feat2398,feat2399,feat2400,Param
0,0_P,45.1,0.203683,0.203992,0.202177,0.205636,0.210894,0.220505,0.226073,0.227012,0.228581,0.230678,0.233982,0.238588,0.241646,0.250014,0.258405,0.261899,0.266396,0.270737,0.274890,0.273968,0.276430,0.283327,0.295028,0.308494,0.316209,0.320717,0.327333,0.331691,0.337992,0.342740,0.349642,0.355668,0.360805,0.364883,0.367570,0.370319,0.376717,0.381299,0.387101,0.393264,0.399336,0.402669,0.405688,0.407629,0.411576,0.416467,0.419428,0.419526,...,1.095705,1.129028,1.153727,1.191501,1.206518,1.240862,1.266370,1.281632,1.303487,1.320699,1.343543,1.349235,1.355927,1.355282,1.360762,1.357333,1.372652,1.387570,1.400359,0.574390,0.584549,0.670901,0.769619,0.920841,1.042641,1.146396,1.211927,1.273028,1.312649,1.513034,1.593172,1.678123,1.751818,1.816820,1.870275,1.911317,1.919118,1.924227,1.962802,0.805311,0.936756,1.290456,1.626534,1.777969,2.094892,2.382082,2.567942,2.697733,2.733829,P
1,1_P,44.8,0.250603,0.249785,0.246786,0.250632,0.255035,0.265170,0.270217,0.269648,0.269498,0.270211,0.272503,0.276662,0.279905,0.288868,0.297204,0.301549,0.306996,0.313169,0.319590,0.320913,0.326546,0.336216,0.350023,0.364886,0.372914,0.377352,0.383670,0.387081,0.392847,0.396461,0.401595,0.405064,0.406857,0.406138,0.403846,0.402338,0.404513,0.405366,0.408354,0.412235,0.416498,0.418540,0.420223,0.421406,0.424962,0.428652,0.430207,0.429708,...,1.502146,1.758616,2.011209,2.160844,2.194579,2.249652,2.285842,2.307155,2.331671,2.351600,2.378405,2.375424,2.375780,2.363581,2.404702,2.419109,2.419744,2.425575,2.429126,0.706865,0.714469,0.781618,0.895926,1.084361,1.143641,1.187382,1.231635,1.280067,1.232515,1.738577,2.511333,3.038777,3.175525,3.269756,3.328903,3.365557,3.357815,3.421406,3.428380,0.992393,1.105412,1.483055,1.721859,1.699395,2.483675,4.277375,4.606743,4.764096,4.809254,P
2,2_P,44.4,0.191200,0.189831,0.187725,0.191629,0.196252,0.205140,0.209914,0.210256,0.210999,0.211794,0.214068,0.218355,0.221650,0.229133,0.235590,0.238543,0.242081,0.245498,0.248445,0.247065,0.249292,0.254473,0.263325,0.273994,0.279563,0.283124,0.288288,0.291484,0.296931,0.301143,0.306834,0.311672,0.315907,0.318279,0.319126,0.320443,0.324220,0.326946,0.331071,0.335602,0.340231,0.343282,0.345376,0.346862,0.350116,0.353510,0.355157,0.355238,...,0.868626,0.886784,0.897235,0.921281,0.942381,0.972106,0.993024,1.004513,1.016845,1.026514,1.040119,1.028430,1.024554,1.021568,1.045355,1.059899,1.062368,1.069155,1.073775,0.539424,0.545641,0.616830,0.695538,0.813433,0.904103,0.975803,1.019546,1.059941,1.061241,1.211937,1.250024,1.300998,1.373126,1.422106,1.455332,1.458446,1.450438,1.497318,1.510940,0.756372,0.862336,1.140623,1.386935,1.473936,1.671600,1.852783,2.015047,2.058019,2.104777,P


## Remove Most correlated??

In [10]:
Special_cols1 = ['Target','Value', 'Param']

In [11]:
"""%%time
num_feature = [col for col in train.columns if col not in Special_cols1]
drop_columns=[]
corr = train[num_feature].corr()
# Drop highly correlated features
columns = np.full((corr.shape[0],), True, dtype=bool)

for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >=0.95 :
            if columns[j]:
                columns[j] = False
                print('FEAT_A: {} FEAT_B: {} - Correlation: {}'.format(train[num_feature].columns[i] , train[num_feature].columns[j], corr.iloc[i,j]))
        elif corr.iloc[i,j] <= -0.95:
            if columns[j]:
                columns[j] = False

drop_columns = train[num_feature].columns[columns == False].values
print('drop_columns',len(drop_columns),drop_columns)

train.drop(drop_columns,inplace=True,axis =1)
print(train.shape)
test.drop(drop_columns,inplace=True,axis =1)
print(test.shape)"""

"%%time\nnum_feature = [col for col in train.columns if col not in Special_cols1]\ndrop_columns=[]\ncorr = train[num_feature].corr()\n# Drop highly correlated features\ncolumns = np.full((corr.shape[0],), True, dtype=bool)\n\nfor i in range(corr.shape[0]):\n    for j in range(i+1, corr.shape[0]):\n        if corr.iloc[i,j] >=0.95 :\n            if columns[j]:\n                columns[j] = False\n                print('FEAT_A: {} FEAT_B: {} - Correlation: {}'.format(train[num_feature].columns[i] , train[num_feature].columns[j], corr.iloc[i,j]))\n        elif corr.iloc[i,j] <= -0.95:\n            if columns[j]:\n                columns[j] = False\n\ndrop_columns = train[num_feature].columns[columns == False].values\nprint('drop_columns',len(drop_columns),drop_columns)\n\ntrain.drop(drop_columns,inplace=True,axis =1)\nprint(train.shape)\ntest.drop(drop_columns,inplace=True,axis =1)\nprint(test.shape)"

In [12]:
test.columns

Index(['feat1', 'feat2', 'feat3', 'feat4', 'feat5', 'feat6', 'feat7', 'feat8',
       'feat9', 'feat10',
       ...
       'feat2394', 'feat2395', 'feat2396', 'feat2397', 'feat2398', 'feat2399',
       'feat2400', 'Target', 'mean_norm', 'Param'],
      dtype='object', length=2403)

## Select Important Features Only?

In [13]:
# From https://github.com/abhishekkrthakur/approachingalmost
from sklearn.feature_selection import f_regression, mutual_info_regression, SelectPercentile
class UnivariateFeatureSelction:
    def __init__(self, n_features, problem_type, scoring, return_cols=True):
        """
        Custom univariate feature selection wrapper on
        different univariate feature selection models from
        scikit-learn.
        :param n_features: SelectPercentile if float else SelectKBest
        :param problem_type: classification or regression
        :param scoring: scoring function, string
        """
        self.n_features = n_features

        if problem_type == "classification":
            valid_scoring = {
                "f_classif": f_classif,
                "chi2": chi2,
                "mutual_info_classif": mutual_info_classif
            }
        else:
            valid_scoring = {
                "f_regression": f_regression,
                "mutual_info_regression": mutual_info_regression
            }
        if scoring not in valid_scoring:
            raise Exception("Invalid scoring function")

        if isinstance(n_features, int):
            self.selection = SelectKBest(
                valid_scoring[scoring],
                k=n_features
            )
        elif isinstance(n_features, float):
            self.selection = SelectPercentile(
                valid_scoring[scoring],
                percentile=int(n_features * 100)
            )
        else:
            raise Exception("Invalid type of feature")

    def fit(self, X, y):
        return self.selection.fit(X, y)

    def transform(self, X):
        return self.selection.transform(X)

    def fit_transform(self, X, y):
        return self.selection.fit_transform(X, y)

    def return_cols(self, X):
        if isinstance(self.n_features, int):
            mask = SelectKBest.get_support(self.selection)
            selected_features = []
            features = list(X.columns)
            for bool, feature in zip(mask, features):
                if bool:
                    selected_features.append(feature)

        elif isinstance(self.n_features, float):
            mask = SelectPercentile.get_support(self.selection)
            selected_features = []
            features = list(X.columns)
            for bool, feature in zip(mask, features):
                if bool:
                    selected_features.append(feature)
        else:
            raise Exception("Invalid type of feature")

        return selected_features

In [14]:
datasets_train = [
    train[train['Param'] == "P"],
    train[train['Param'] == "K"],
    train[train['Param'] == "Mg"],
    train[train['Param'] == "pH"],
]

datasets_test = [
    test[test['Param'] == "P"],
    test[test['Param'] == "K"],
    test[test['Param'] == "Mg"],
    test[test['Param'] == "pH"],
]

# BASELINE MODEL PERFORMANCE

In [15]:
#selected_features_dict = {}  # Create an empty dictionary to store selected features for each dataset
dataset=train[train['Param'] == "P"]
X = dataset.drop(columns=["Target", "Param", "Value"])
y = dataset["Value"]

"""ufs = UnivariateFeatureSelction(
    n_features=0.9,  # Select 90% of the most important features
    problem_type="regression",
    scoring="f_regression"
)

ufs.fit(X, y.values.ravel())
selected_features = ufs.return_cols(X)"""
selected_features = X.columns
#print(f"Selected features for P: {selected_features}")

#selected_features_dict[dataset_name] = selected_features  # Store selected features in the dictionary

# Now selected_features_dict contains selected features for each dataset
# You can access them using dataset names as keys
X=X[selected_features]

In [16]:
def train_and_evaluate_model(X_train, y_train, X_test, y_test, selected_columns=None):
    if selected_columns is not None:
        # If selected columns are provided, use only those columns
        X_train = X_train[selected_columns]
        X_test = X_test[selected_columns]

    # Train and evaluate the model
    # (Rest of the code remains unchanged)
    model = HistGradientBoostingRegressor(random_state=1234, verbose=0)
    model_name = "LGBMoost"
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = mean_squared_error(y_test, preds, squared=False)

    return model, model_name, rmse

best_model = None
best_rmse = float('inf')
results_per_dataset = []
best_models_per_dataset = []
# Initialize a KFold with the number of splits
n_splits = 5  # You can adjust this as needed
kfold = KFold(n_splits=n_splits)
results = []
for train_idx, test_idx in kfold.split(X):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]

    model, model_name, rmse = train_and_evaluate_model(X_train, y_train, X_test, y_test, selected_features)
    results.append([model_name, rmse])

    # Check if this model has a lower RMSE than the best model so far
    if rmse < best_rmse:
        best_model = model
        best_rmse = rmse

results_per_dataset.append(results)
best_models_per_dataset.append(best_model)

# Print results for each dataset
for i, best_model in enumerate(best_models_per_dataset):
    print(f"Best Model for Dataset {i}: {best_model}")

# Print results for each dataset
for i, results in enumerate(results_per_dataset):
    print(f"Results for Dataset {i}:")
    results_df = pd.DataFrame(results, columns=["Model", "RMSE"])
    print(tabulate(results_df, headers="keys", tablefmt="fancy_grid"))

Best Model for Dataset 0: HistGradientBoostingRegressor(random_state=1234)
Results for Dataset 0:
╒════╤══════════╤═════════╕
│    │ Model    │    RMSE │
╞════╪══════════╪═════════╡
│  0 │ LGBMoost │ 22.2547 │
├────┼──────────┼─────────┤
│  1 │ LGBMoost │ 15.1639 │
├────┼──────────┼─────────┤
│  2 │ LGBMoost │ 14.4393 │
├────┼──────────┼─────────┤
│  3 │ LGBMoost │ 17.7383 │
├────┼──────────┼─────────┤
│  4 │ LGBMoost │ 13.2555 │
╘════╧══════════╧═════════╛


In [17]:
# Make predictions on the Zindi test set
model=best_models_per_dataset[0]
test_df = datasets_test[0][selected_features]
preds = model.predict(test_df) #.fillna(0)

# Create submisiion file to be uploaded to Zindi for scoring
sub_k = pd.DataFrame({'sample_index': datasets_test[0]['Target'], 'Target': preds})
sub_k['Target']=sub_k['Target']/70.3026558891455
sub_k.head()

,sample_index,Target
0,0_P,1.144516
1,1_P,1.077322
2,2_P,0.896702
3,3_P,1.102193
4,4_P,0.853410


# K

In [18]:
#selected_features_dict = {}  # Create an empty dictionary to store selected features for each dataset
dataset=train[train['Param'] == "K"]
X = dataset.drop(columns=["Target", "Param", "Value"])
y = dataset["Value"]
"""
ufs = UnivariateFeatureSelction(
    n_features=0.9,  # Select 90% of the most important features
    problem_type="regression",
    scoring="f_regression"
)

ufs.fit(X, y.values.ravel())
selected_features = ufs.return_cols(X)"""
selected_features = X.columns

#print(f"Selected features for P: {selected_features}")

#selected_features_dict[dataset_name] = selected_features  # Store selected features in the dictionary

# Now selected_features_dict contains selected features for each dataset
# You can access them using dataset names as keys
X=X[selected_features]

In [19]:
def train_and_evaluate_model(X_train, y_train, X_test, y_test, selected_columns=None):
    if selected_columns is not None:
        # If selected columns are provided, use only those columns
        X_train = X_train[selected_columns]
        X_test = X_test[selected_columns]

    # Train and evaluate the model
    # (Rest of the code remains unchanged)
    model = HistGradientBoostingRegressor(random_state=1234, verbose=0)
    model_name = "LGBMoost"
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = mean_squared_error(y_test, preds, squared=False)

    return model, model_name, rmse

best_model = None
best_rmse = float('inf')
results_per_dataset = []
best_models_per_dataset = []
# Initialize a KFold with the number of splits
n_splits = 5  # You can adjust this as needed
kfold = KFold(n_splits=n_splits)
results = []
for train_idx, test_idx in kfold.split(X):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]

    model, model_name, rmse = train_and_evaluate_model(X_train, y_train, X_test, y_test, selected_features)
    results.append([model_name, rmse])

    # Check if this model has a lower RMSE than the best model so far
    if rmse < best_rmse:
        best_model = model
        best_rmse = rmse

results_per_dataset.append(results)
best_models_per_dataset.append(best_model)

# Print results for each dataset
for i, best_model in enumerate(best_models_per_dataset):
    print(f"Best Model for Dataset {i}: {best_model}")

# Print results for each dataset
for i, results in enumerate(results_per_dataset):
    print(f"Results for Dataset {i}:")
    results_df = pd.DataFrame(results, columns=["Model", "RMSE"])
    print(tabulate(results_df, headers="keys", tablefmt="fancy_grid"))

Best Model for Dataset 0: HistGradientBoostingRegressor(random_state=1234)
Results for Dataset 0:
╒════╤══════════╤═════════╕
│    │ Model    │    RMSE │
╞════╪══════════╪═════════╡
│  0 │ LGBMoost │ 33.5658 │
├────┼──────────┼─────────┤
│  1 │ LGBMoost │ 40.9432 │
├────┼──────────┼─────────┤
│  2 │ LGBMoost │ 27.4212 │
├────┼──────────┼─────────┤
│  3 │ LGBMoost │ 39.9622 │
├────┼──────────┼─────────┤
│  4 │ LGBMoost │ 35.598  │
╘════╧══════════╧═════════╛


In [20]:
# Make predictions on the Zindi test set
model=best_models_per_dataset[0]
test_df = datasets_test[1][selected_features]
preds = model.predict(test_df) #.fillna(1)
# Create submisiion file to be uploaded to Zindi for scoring
sub_p = pd.DataFrame({'sample_index': datasets_test[1]['Target'], 'Target': preds})
sub_p['Target']=sub_p['Target']/227.9885103926097
#sub_p['Target']=sub_p['Target']/234.18273333
sub_p.head()

,sample_index,Target
1154,0_K,0.891158
1155,1_K,0.988521
1156,2_K,1.055790
1157,3_K,1.015227
1158,4_K,0.905135


# Mg

In [21]:
#selected_features_dict = {}  # Create an empty dictionary to store selected features for each dataset
dataset=train[train['Param'] == "Mg"]
X = dataset.drop(columns=["Target", "Param", "Value"])
y = dataset["Value"]

"""ufs = UnivariateFeatureSelction(
    n_features=0.9,  # Select 90% of the most important features
    problem_type="regression",
    scoring="f_regression"
)

ufs.fit(X, y.values.ravel())
selected_features = ufs.return_cols(X)"""

selected_features = X.columns
#print(f"Selected features for P: {selected_features}")

#selected_features_dict[dataset_name] = selected_features  # Store selected features in the dictionary

# Now selected_features_dict contains selected features for each dataset
# You can access them using dataset names as keys
X=X[selected_features]

In [22]:
def train_and_evaluate_model(X_train, y_train, X_test, y_test, selected_columns=None):
    if selected_columns is not None:
        # If selected columns are provided, use only those columns
        X_train = X_train[selected_columns]
        X_test = X_test[selected_columns]

    # Train and evaluate the model
    # (Rest of the code remains unchanged)
    model = HistGradientBoostingRegressor(random_state=1234, verbose=0)
    model_name = "LGBMoost"
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = mean_squared_error(y_test, preds, squared=False)

    return model, model_name, rmse

best_model = None
best_rmse = float('inf')
results_per_dataset = []
best_models_per_dataset = []
# Initialize a KFold with the number of splits
n_splits = 5  # You can adjust this as needed
kfold = KFold(n_splits=n_splits)
results = []
for train_idx, test_idx in kfold.split(X):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]

    model, model_name, rmse = train_and_evaluate_model(X_train, y_train, X_test, y_test, selected_features)
    results.append([model_name, rmse])

    # Check if this model has a lower RMSE than the best model so far
    if rmse < best_rmse:
        best_model = model
        best_rmse = rmse

results_per_dataset.append(results)
best_models_per_dataset.append(best_model)

# Print results for each dataset
for i, best_model in enumerate(best_models_per_dataset):
    print(f"Best Model for Dataset {i}: {best_model}")

# Print results for each dataset
for i, results in enumerate(results_per_dataset):
    print(f"Results for Dataset {i}:")
    results_df = pd.DataFrame(results, columns=["Model", "RMSE"])
    print(tabulate(results_df, headers="keys", tablefmt="fancy_grid"))

Best Model for Dataset 0: HistGradientBoostingRegressor(random_state=1234)
Results for Dataset 0:
╒════╤══════════╤═════════╕
│    │ Model    │    RMSE │
╞════╪══════════╪═════════╡
│  0 │ LGBMoost │ 29.8413 │
├────┼──────────┼─────────┤
│  1 │ LGBMoost │ 14.6883 │
├────┼──────────┼─────────┤
│  2 │ LGBMoost │ 22.7145 │
├────┼──────────┼─────────┤
│  3 │ LGBMoost │ 18.4412 │
├────┼──────────┼─────────┤
│  4 │ LGBMoost │ 15.2905 │
╘════╧══════════╧═════════╛


In [23]:
# Make predictions on the Zindi test set
model=best_models_per_dataset[0]
test_df = datasets_test[2][selected_features]
preds = model.predict(test_df) #.fillna(0)

# Create submisiion file to be uploaded to Zindi for scoring
sub_mg = pd.DataFrame({'sample_index': datasets_test[2]['Target'], 'Target': preds})
sub_mg['Target']=sub_mg['Target']/159.28123556581986
sub_mg.head()

,sample_index,Target
2308,0_Mg,0.762736
2309,1_Mg,0.933107
2310,2_Mg,1.095319
2311,3_Mg,1.193688
2312,4_Mg,1.004106


# pH

In [24]:
#selected_features_dict = {}  # Create an empty dictionary to store selected features for each dataset
dataset=train[train['Param'] == "pH"]
X = dataset.drop(columns=["Target", "Param", "Value"])
y = dataset["Value"]

"""ufs = UnivariateFeatureSelction(
    n_features=0.9,  # Select 90% of the most important features
    problem_type="regression",
    scoring="f_regression"
)

ufs.fit(X, y.values.ravel())
selected_features = ufs.return_cols(X)"""

selected_features = X.columns

#print(f"Selected features for P: {selected_features}")

#selected_features_dict[dataset_name] = selected_features  # Store selected features in the dictionary

# Now selected_features_dict contains selected features for each dataset
# You can access them using dataset names as keys
X=X[selected_features]

In [25]:
def train_and_evaluate_model(X_train, y_train, X_test, y_test, selected_columns=None):
    if selected_columns is not None:
        # If selected columns are provided, use only those columns
        X_train = X_train[selected_columns]
        X_test = X_test[selected_columns]

    # Train and evaluate the model
    # (Rest of the code remains unchanged)
    model = HistGradientBoostingRegressor(random_state=1234, verbose=0)
    model_name = "LGBMoost"
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = mean_squared_error(y_test, preds, squared=False)

    return model, model_name, rmse

best_model = None
best_rmse = float('inf')
results_per_dataset = []
best_models_per_dataset = []
# Initialize a KFold with the number of splits
n_splits = 5  # You can adjust this as needed
kfold = KFold(n_splits=n_splits)
results = []
for train_idx, test_idx in kfold.split(X):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]

    model, model_name, rmse = train_and_evaluate_model(X_train, y_train, X_test, y_test, selected_features)
    results.append([model_name, rmse])

    # Check if this model has a lower RMSE than the best model so far
    if rmse < best_rmse:
        best_model = model
        best_rmse = rmse

results_per_dataset.append(results)
best_models_per_dataset.append(best_model)

# Print results for each dataset
for i, best_model in enumerate(best_models_per_dataset):
    print(f"Best Model for Dataset {i}: {best_model}")

# Print results for each dataset
for i, results in enumerate(results_per_dataset):
    print(f"Results for Dataset {i}:")
    results_df = pd.DataFrame(results, columns=["Model", "RMSE"])
    print(tabulate(results_df, headers="keys", tablefmt="fancy_grid"))

Best Model for Dataset 0: HistGradientBoostingRegressor(random_state=1234)
Results for Dataset 0:
╒════╤══════════╤══════════╕
│    │ Model    │     RMSE │
╞════╪══════════╪══════════╡
│  0 │ LGBMoost │ 0.16585  │
├────┼──────────┼──────────┤
│  1 │ LGBMoost │ 0.142672 │
├────┼──────────┼──────────┤
│  2 │ LGBMoost │ 0.131732 │
├────┼──────────┼──────────┤
│  3 │ LGBMoost │ 0.141275 │
├────┼──────────┼──────────┤
│  4 │ LGBMoost │ 0.135748 │
╘════╧══════════╧══════════╛


In [26]:
# Make predictions on the Zindi test set
model=best_models_per_dataset[0]
test_df = datasets_test[3][selected_features]
preds = model.predict(test_df) #.fillna(0)

# Create submisiion file to be uploaded to Zindi for scoring
sub_ph = pd.DataFrame({'sample_index': datasets_test[3]['Target'], 'Target': preds})
sub_ph['Target']=sub_ph['Target']/6.782719399538106
sub_ph.head()

,sample_index,Target
3462,0_pH,1.017651
3463,1_pH,1.014242
3464,2_pH,1.004131
3465,3_pH,1.004610
3466,4_pH,1.010353


In [27]:
sub=pd.concat([sub_p,sub_k,sub_mg,sub_ph])
sub.to_csv('HIST.csv', index = False)
print(sub.shape)
sub#.head()

(4616, 2)


,sample_index,Target
1154,0_K,0.891158
1155,1_K,0.988521
1156,2_K,1.055790
1157,3_K,1.015227
1158,4_K,0.905135
...,...,...
4611,1149_pH,0.971884
4612,1150_pH,0.959529
4613,1151_pH,0.972270
4614,1152_pH,0.963777
